In [2]:
import pickle
from prettytable import PrettyTable
import matplotlib.pyplot as plt
def load_data(filename):
    f = open(filename, 'rb')
    loaded_data = pickle.load(f)
    f.close()
    return loaded_data
    
def get_values(val_score):
    M_fpr = [i["M_fpr"] for i in val_score]
    M_fnr = [i["M_fnr"] for i in val_score]
    M_f1 = [i["M_f1"] for i in val_score]
    W_fpr = [i["W_fpr"] for i in val_score]
    W_fnr = [i["W_fnr"] for i in val_score]
    W_f1 = [i["W_f1"] for i in val_score]
    ACC = [i["ACC"] for i in val_score]
    return [M_fpr, M_fnr, M_f1, W_fpr, W_fnr, W_f1, ACC]

def get_values_both(train_core, val_score):
    return (get_values(train_core), get_values(val_score))
    

In [2]:
def draw(score, x, x_start, x_end, value_dict):
    for i, value  in enumerate(get_values(score)):
        value = [float(i) for i in value]
        plt.plot(x, value, label='-' + value_dict[i])
        plt.xlim(x_start, x_end)
        plt.ylabel("testing accuracy %")
        plt.xlabel("epoch (iter. over training set)")
        plt.legend(loc="lower right", fontsize='small', ncol=2,)
        plt.show()
def draw_train_and_test(train_score, val_score, x, x_start, x_end, value_dict):
    for i, value  in enumerate(get_values_both(train_score, val_score)):
        value_train = [float(i) for i in value[0]]
        value_val = [float(i) for i in value[1]]
        # print(value)
        # print(sum(value)/len(value))
        plt.plot(x, value_train, label='-train-' + value_dict[i])
        plt.plot(x, value_val, label='-val-' + value_dict[i])
        plt.xlim(x_start, x_end)
        plt.ylabel("testing accuracy %")
        plt.xlabel("epoch (iter. over training set)")
        plt.legend(loc="lower right", fontsize='small', ncol=2,)
        plt.show()

In [3]:
def show_table(savename):
    fold_result_dict = {}
    filename = "./save/result/" + savename + ".result"
    fold_result_dict = load_data(filename)
    train_table = PrettyTable(['typ', 'epo', 'loss', 'M_fpr', 'M_fnr', 'M_f1', 'W_fpr', 'W_fnr', 'W_f1', 'ACC'])
    test_table = PrettyTable(['typ', 'epo', 'loss', 'M_fpr', 'M_fnr', 'M_f1', 'W_fpr', 'W_fnr', 'W_f1', 'ACC'])
    # print(fold_result_dict)
    for i in fold_result_dict:
        train_table.add_row(["tra", str(i+1), format(fold_result_dict[i]["train_loss"], '.4f')] + \
            [fold_result_dict[i]["train_score"][j] for j in fold_result_dict[i]["train_score"] if j != "MCM"])
        test_table.add_row(["val", str(i+1), format(fold_result_dict[i]["val_loss"], '.4f')] + \
            [fold_result_dict[i]["val_score"][j] for j in fold_result_dict[i]["val_score"] if j != "MCM"])
        # if i == 39: break
    max_mf1 =  max([fold_result_dict[i]["val_score"]["M_f1"] for i in fold_result_dict])
    mean_mf1 =  sum([float(fold_result_dict[i]["val_score"]["M_f1"]) for i in fold_result_dict][-3:])/3
    print(filename)
    print(train_table)
    print(test_table)
    print(max_mf1)
    print(mean_mf1)
    return max_mf1, mean_mf1
    

def draw_table(savename):
    value_dict = ["M_fpr", "M_fnr", "M_f1", "W_fpr", "W_fnr", "W_f1", "ACC"]
    x_start = 0
    step = 1
    fold_result_dict = {}
    filename = "./save/result/" + savename + ".result"
    fold_result_dict = load_data(filename)

    x_end = x_start + step * len(fold_result_dict)
    x = range(x_start,x_end,step)
    # x = range(10,x_end,step)
    # x = range(40,x_end,step)

    train_loss = [fold_result_dict[i]['train_loss'] for i in x]
    val_loss = [fold_result_dict[i]['val_loss'] for i in x]
    plt.figure()
    plt.plot(x, train_loss, label='-train_loss')
    plt.plot(x, val_loss, label='-val_loss')
    plt.xlim(x_start, x_end)
    plt.ylabel("testing accuracy %")
    plt.xlabel("epoch (iter. over training set)")
    plt.legend(loc="lower right", fontsize='small', ncol=2,)
    plt.show()
    
    train_score = [fold_result_dict[i]['train_score'] for i in x]
    val_score = [fold_result_dict[i]['val_score'] for i in x]
    # draw_train_and_test(train_score, val_score, x, x_start, x_end, value_dict)
    draw(train_score, x, x_start, x_end, value_dict)
    draw(val_score, x, x_start, x_end, value_dict)
def evaluate(savename):
    max_mf1, mean_mf1 = show_table(savename)
    # draw_table(savename)
    return max_mf1, mean_mf1


In [13]:
# 无监督数据集评价
evaluate("Unsuper_sard_nr_2_bat32_lam0.9_tem0.1")
evaluate("Unsuper_real_nr_2_bat32_lam0.9_tem0.1")

./save/result/Unsuper_sard_nr_2_bat32_lam0.9_tem0.1.result
+-----+-----+--------+--------+--------+--------+--------+--------+--------+--------+
| typ | epo |  loss  | M_fpr  | M_fnr  |  M_f1  | W_fpr  | W_fnr  |  W_f1  |  ACC   |
+-----+-----+--------+--------+--------+--------+--------+--------+--------+--------+
| tra |  1  | 0.4893 | 48.978 | 48.978 | 50.802 | 49.458 | 48.499 | 51.870 | 51.501 |
| tra |  2  | 0.1185 | 47.763 | 47.763 | 51.878 | 47.923 | 47.603 | 52.788 | 52.397 |
| tra |  3  | 0.0917 | 46.190 | 46.190 | 53.349 | 46.163 | 46.217 | 54.169 | 53.783 |
| tra |  4  | 0.0814 | 46.491 | 46.491 | 53.120 | 46.608 | 46.374 | 54.007 | 53.626 |
| tra |  5  | 0.0718 | 45.551 | 45.551 | 54.069 | 45.701 | 45.402 | 54.967 | 54.598 |
| tra |  6  | 0.0721 | 46.164 | 46.164 | 53.407 | 46.202 | 46.126 | 54.256 | 53.874 |
| tra |  7  | 0.0670 | 45.613 | 45.613 | 53.946 | 45.635 | 45.591 | 54.787 | 54.409 |
| tra |  8  | 0.0675 | 45.209 | 45.209 | 54.359 | 45.254 | 45.163 | 55.209 | 54.8

('51.771', 50.81133333333333)

In [14]:
# 整体数据集评价

# evaluate("CE_realdata_2_bat32")  
# evaluate("Contras_realdata_2_bat32_lam0.9_tem0.1") 

# evaluate("CE_real_nr_2_bat32") 
# evaluate("Contras_real_nr_2_bat32_lam0.9_tem0.1") 
# evaluate("CE_real_nr_2_sub_bat32")  
# evaluate("Contras_real_nr_2_sub_bat32_lam0.9_tem0.1")   

# 没有这个
# evaluate("CE_sard_2_bat32")  
# evaluate("Contras_sard_2_bat32_lam0.9_tem0.1")  
# evaluate("CE_sard_2_sub_bat32")  
# evaluate("Contras_sard_2_sub_bat32_lam0.9_tem0.1") 

# evaluate("CE_sard_nr_2_bat32")  
# evaluate("Contras_sard_nr_2_bat32_lam0.9_tem0.1")  
# evaluate("CE_sard_nr_2_sub_bat32")  
# evaluate("Contras_sard_nr_2_sub_bat32_lam0.9_tem0.1")  



./save/result/CE_sard_nr_2_bat32.result
+-----+-----+--------+-------+-------+--------+-------+-------+--------+--------+
| typ | epo |  loss  | M_fpr | M_fnr |  M_f1  | W_fpr | W_fnr |  W_f1  |  ACC   |
+-----+-----+--------+-------+-------+--------+-------+-------+--------+--------+
| tra |  1  | 0.1662 | 9.492 | 9.492 | 90.325 | 9.590 | 9.395 | 90.625 | 90.605 |
| tra |  2  | 0.0659 | 3.625 | 3.625 | 95.960 | 3.312 | 3.939 | 96.076 | 96.061 |
| tra |  3  | 0.0619 | 3.538 | 3.538 | 96.151 | 3.331 | 3.745 | 96.266 | 96.255 |
| tra |  4  | 0.0600 | 3.712 | 3.712 | 96.044 | 3.579 | 3.844 | 96.165 | 96.156 |
| tra |  5  | 0.0569 | 3.380 | 3.380 | 96.377 | 3.239 | 3.520 | 96.488 | 96.480 |
| tra |  6  | 0.0565 | 3.410 | 3.410 | 96.432 | 3.358 | 3.462 | 96.544 | 96.538 |
| tra |  7  | 0.0563 | 3.385 | 3.385 | 96.425 | 3.300 | 3.471 | 96.536 | 96.529 |
| tra |  8  | 0.0560 | 3.308 | 3.308 | 96.525 | 3.245 | 3.372 | 96.634 | 96.628 |
| tra |  9  | 0.0558 | 3.315 | 3.315 | 96.511 | 3.244 | 3.

('96.822', 96.25)

In [8]:
# small 数据集评价
train_list = [20,50,100,500,1000]
train_list = [100]
seed_num = 10
batch_size = 32

sub_bool = False
sub = "" if sub_bool == False else "sub_"

for train_num in train_list:
    batch_size = 16 if train_num == 20 else 32
    for seed in range(seed_num):
        print(seed)
        evaluate("CE_real_nr_2_" + sub + "bat" + str(batch_size) + "_seed" + str(seed) + "_" + str(train_num) + "_500")
        evaluate("Contras_real_nr_2_" + sub + "bat" + str(batch_size) + "_lam0.9_tem0.1_seed" + str(seed) + "_" + str(train_num) + "_500")
        print("-"*100)
        print()
print("="*40)

for train_num in train_list:
    batch_size = 16 if train_num == 20 else 32
    for seed in range(seed_num):
        print(seed)
        evaluate("CE_sard_nr_2_" + sub + "bat" + str(batch_size) + "_seed" + str(seed) + "_" + str(train_num) + "_500")
        evaluate("Contras_sard_nr_2_" + sub + "bat" + str(batch_size) + "_lam0.9_tem0.1_seed" + str(seed) + "_" + str(train_num) + "_500")
        print("-"*100)
        print()
print("*" * 200)

0
./save/result/CE_real_nr_2_bat32_seed0_100_500.result
+-----+-----+--------+--------+--------+---------+--------+--------+---------+---------+
| typ | epo |  loss  | M_fpr  | M_fnr  |   M_f1  | W_fpr  | W_fnr  |   W_f1  |   ACC   |
+-----+-----+--------+--------+--------+---------+--------+--------+---------+---------+
| tra |  1  | 0.6975 | 57.000 | 57.000 |  38.518 | 57.000 | 57.000 |  38.518 |  57.000 |
| tra |  2  | 0.6996 | 52.000 | 52.000 |  32.432 | 52.000 | 52.000 |  32.432 |  52.000 |
| tra |  3  | 0.6951 | 46.000 | 46.000 |  53.926 | 46.000 | 46.000 |  53.926 |  54.000 |
| tra |  4  | 0.6688 | 34.000 | 34.000 |  62.637 | 34.000 | 34.000 |  62.637 |  66.000 |
| tra |  5  | 0.6311 | 35.000 | 35.000 |  62.667 | 35.000 | 35.000 |  62.667 |  65.000 |
| tra |  6  | 0.5114 | 19.000 | 19.000 |  80.952 | 19.000 | 19.000 |  80.952 |  81.000 |
| tra |  7  | 0.4487 | 10.000 | 10.000 |  89.899 | 10.000 | 10.000 |  89.899 |  90.000 |
| tra |  8  | 0.3272 | 18.000 | 18.000 |  81.397 | 18.

In [19]:
# kfold 数据集评价

filename = "mvd_nr_40"
# filename = "mvd_nr_40_nsr"
result = {"ce_max":[], "ce_mean": [], "contras_max": [], "contras_mean": []  }
for fold_num in range(8):
    save_name_ce = "CE_Kfold_" + str(fold_num) + "_" + filename + "_bat32"
    save_name_contras = "Contras_Kfold_" + str(fold_num) + "_" + filename + "_bat32_lam0.9_tem0.1"
    max_mf1_ce, mean_mf1_ce = evaluate(save_name_ce)
    max_mf1_contras, mean_mf1_contras = evaluate(save_name_contras)
    result["ce_max"].append(max_mf1_ce)
    result["ce_mean"].append(mean_mf1_ce)
    result["contras_max"].append(max_mf1_contras)
    result["contras_mean"].append(mean_mf1_contras)

print(result)
print(sum([float(i) for i in result["ce_max"]])/len(result["ce_max"]))
print(sum([float(i) for i in result["ce_mean"]])/len(result["ce_mean"]))
print(sum([float(i) for i in result["contras_max"]])/len(result["contras_max"]))
print(sum([float(i) for i in result["contras_mean"]])/len(result["contras_mean"]))

./save/result/CE_Kfold_0_mvd_nr_40_bat32.result
88.689
83.482
./save/result/Contras_Kfold_0_mvd_nr_40_bat32_lam0.9_tem0.1.result
89.318
87.969
./save/result/CE_Kfold_1_mvd_nr_40_bat32.result
93.977
90.508
./save/result/Contras_Kfold_1_mvd_nr_40_bat32_lam0.9_tem0.1.result
92.724
88.07966666666665
./save/result/CE_Kfold_2_mvd_nr_40_bat32.result
84.982
83.598
./save/result/Contras_Kfold_2_mvd_nr_40_bat32_lam0.9_tem0.1.result
85.459
82.88133333333333
./save/result/CE_Kfold_3_mvd_nr_40_bat32.result
92.027
92.027
./save/result/Contras_Kfold_3_mvd_nr_40_bat32_lam0.9_tem0.1.result
93.390
90.295
./save/result/CE_Kfold_4_mvd_nr_40_bat32.result
89.641
89.363
./save/result/Contras_Kfold_4_mvd_nr_40_bat32_lam0.9_tem0.1.result
88.756
88.012
./save/result/CE_Kfold_5_mvd_nr_40_bat32.result
89.445
88.891
./save/result/Contras_Kfold_5_mvd_nr_40_bat32_lam0.9_tem0.1.result
86.205
85.574
./save/result/CE_Kfold_6_mvd_nr_40_bat32.result
87.058
84.39933333333333
./save/result/Contras_Kfold_6_mvd_nr_40_bat32_l

In [7]:
# kfold 数据集评价

# filename = "mvd_nr_40_nsr"
filename = "mvd_allkfold_40"
for fold_num in range(10):
    save_name_ce = "CE_Kfold_" + str(fold_num) + "_" + filename + "_bat32"
    evaluate(save_name_ce)
    print()
    print("**"*50)
    print()
    # for lam in [0.1, 0.3, 0.5, 0.7, 0.9, 1.0]:
    for lam in [0.1, 0.3, 0.5]:
        # for temperature in [0.1,0.3,0.5, 0.7]:
        for temperature in [0.1,0.3]:
            save_name_contras = "Contras2_Kfold_" + str(fold_num) + "_" + filename + "_bat32_lam" + str(lam) + "_tem" + str(temperature)
            evaluate(save_name_contras)
        print()
        print("**"*50)
        print()
    print()
    print("==="*50)
    print()

./save/result/CE_Kfold_0_mvd_allkfold_40_bat32.result
+-----+-----+--------+-------+--------+--------+-------+-------+--------+--------+
| typ | epo |  loss  | M_fpr | M_fnr  |  M_f1  | W_fpr | W_fnr |  W_f1  |  ACC   |
+-----+-----+--------+-------+--------+--------+-------+-------+--------+--------+
| tra |  1  | 0.4177 | 0.267 | 46.198 | 57.696 | 2.581 | 8.109 | 91.254 | 91.893 |
| tra |  2  | 0.0916 | 0.056 | 23.713 | 78.331 | 0.379 | 1.860 | 97.926 | 98.140 |
| tra |  3  | 0.0518 | 0.036 | 12.959 | 88.858 | 0.296 | 1.153 | 98.776 | 98.847 |
| tra |  4  | 0.0332 | 0.029 | 6.665  | 94.085 | 0.299 | 0.859 | 99.119 | 99.141 |
| tra |  5  | 0.0258 | 0.025 | 4.934  | 95.620 | 0.255 | 0.758 | 99.233 | 99.242 |
| tra |  6  | 0.0206 | 0.024 | 3.929  | 96.374 | 0.258 | 0.686 | 99.307 | 99.314 |
| tra |  7  | 0.0179 | 0.022 | 3.218  | 97.448 | 0.246 | 0.632 | 99.363 | 99.368 |
| tra |  8  | 0.0163 | 0.022 | 2.712  | 97.755 | 0.253 | 0.619 | 99.377 | 99.381 |
| tra |  9  | 0.0148 | 0.021 | 1.

FileNotFoundError: [Errno 2] No such file or directory: './save/result/Contras2_Kfold_0_mvd_allkfold_40_bat32_lam0.5_tem0.1.result'